# QBID under TCJA

## Key Quantities for Section 199A

Let:

- $ \text{QBI} $: Qualified Business Income — net income from a U.S. trade or business
- $ W $: W-2 wages paid
- $ P $: UBIA (Unadjusted Basis Immediately after Acquisition) of qualified property
- $\text{WPA} = \max \left( 0.5 \cdot W,\ 0.25 \cdot W + 0.025 \cdot P \right)$
- $ \theta_{\max} $: maximum deduction rate (typically 0.20)
- $ T $: taxable income  
- $ G $: net capital gain  
- $ \text{MTI} = T - G $: Modified Taxable Income  
- $ \tau $: income threshold  
- $ \ell $: phase-in range  
- $ p = \min \left( 1,\ \max \left( 0,\ \frac{T - \tau}{\ell} \right) \right) $: proportion through the phase-in  



## The Sum Described in (1)(A)

The internal limitation described in Section 199A(b)(2)(B)(i)(I) is:

$$
S^{(1)(A)} = \min \left( \theta_{\max} \cdot \text{QBI},\ \text{WPA} \right)
$$


## non-SSTB Preliminary Deduction (Before MTI Cap)

Let \( D_{\text{pre}} \) be the preliminary deduction before applying the MTI limitation:

$$
D_{\text{pre}} =
\begin{cases}
\theta_{\max} \cdot \text{QBI}, & \text{if } T \leq \tau \\\\[12pt]
\theta_{\max} \cdot \text{QBI} - p \cdot \left( \theta_{\max} \cdot \text{QBI} - \text{WPA} \right), & \text{if } \tau < T < \tau + \ell \\\\[12pt]
\text{WPA}, & \text{if } T \geq \tau + \ell
\end{cases}
$$



## SSTB Preliminary Deduction (Before MTI Cap)

Let \( D_{\text{pre}} \) be the preliminary deduction before applying the MTI limitation:

$$
D_{\text{pre}} =
\begin{cases}
\theta_{\max} \cdot \text{QBI}, & \text{if } T \leq \tau \\\\[12pt]
(1 - p) \cdot \min \left( \theta_{\max} \cdot \text{QBI},\ \text{WPA} \right), & \text{if } \tau < T < \tau + \ell \\\\[12pt]
0, & \text{if } T \geq \tau + \ell
\end{cases}
$$


## Final Deduction (After MTI Cap)

The final deduction is the lesser of the preliminary deduction and the MTI cap:

$$
\text{Deduction} = \min \left( D_{\text{pre}},\ \theta_{\max} \cdot \text{MTI} \right)
$$

# QBID under H.R. 1

In addition to the other parameters, let:

- $r$: ramp-down rate (75¢ of deduction lost per $1 above threshold τ)

---

### Non-SSTB Deduction

**Preliminary deduction (before MTI cap):**

$$
D_{\text{pre}} =
\begin{cases}
\theta_{\max} \cdot \text{QBI}, & \text{if } T \leq \tau \\\\
\max \left( \text{WPA},\ \theta_{\max} \cdot \text{QBI} - r \cdot (T - \tau) \right), & \text{if } T > \tau
\end{cases}
$$

---

### SSTB Deduction

**Preliminary deduction (before MTI cap):**

$$
D_{\text{pre}} =
\begin{cases}
\theta_{\max} \cdot \text{QBI}, & \text{if } T \leq \tau \\\\
\max \left( 0,\ \theta_{\max} \cdot \text{QBI} - r \cdot (T - \tau) \right), & \text{if } T > \tau
\end{cases}
$$

In [ ]:
#